### 二项逻辑斯蒂回归模型

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
#    导入sklearn自带数据集
from sklearn.datasets import load_iris


In [2]:
iris = load_iris()
#    转为DataFrame
irisData = pd.DataFrame(iris.data, columns=iris.feature_names)
irisData['Label'] = iris.target

#    只用两个分类，因此将Label=2的删除
irisData = irisData[irisData['Label'] < 2]

In [3]:
#    将数据转为矩阵形式，并区分feature和label
X = np.array(irisData.iloc[:,:-1])
y = np.array(irisData['Label'])

X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.4, random_state=5)

ones = np.ones(X_train.shape[0])
X_train = np.column_stack((X_train, ones))
ones = np.ones(X_test.shape[0])
X_test = np.column_stack((X_test, ones))

In [4]:
#    初始化参数
b = 0
omiga = np.zeros((1,X.shape[1]))
omiga, b

(array([[0., 0., 0., 0.]]), 0)

In [5]:
#    合并omiga和b赋值新的omiga，同时也要对x合并1赋值新的x
omiga = np.append(omiga, b)
omiga

array([0., 0., 0., 0., 0.])

In [6]:

#    定义二项逻辑斯蒂回归模型
def y_is_1(omiga, x):
    '''
    P(Y=1|x)的概率
    '''
    theta = np.exp(omiga.dot(x))
    return theta/(1+theta)

def y_is_0(omiga, x):
    '''
    P(Y=0|x)的概率
    '''
    theta = np.exp(omiga.dot(x))
    return 1/(1+theta)

                                                                               
     

In [7]:
#    对数似然函数
def L_omiga(omiga, X, y):
    '''
    对数似然函数
    omiga: 扩充的参数向量(w, b)
    X: 扩充的特折矩阵 第i行为(xi, 1)
    y: 标枪 yi表示第i个label
    
    return: 返回对数似然函数的值
    '''
    re = 0
    for i in range(X.shape[0]):
        theta = omiga.dot(X[i])
        re += ((y[i]*theta) - np.log(1 + np.exp(theta)))
    return re




In [8]:
def gradient_L_omiga(omiga, X, y):
    '''
    对数似然函数的梯度
    omiga: 扩充的参数向量(w, b)
    X: 扩充的特折矩阵 第i行为(xi, 1)
    y: 标枪 yi表示第i个label
    
    return: 返回对数似然函数的的梯度
    '''
    re = [0]*omiga.shape[1]
    for i in range(X.shape[0]):
        theta = omiga.dot(X[i])
        re += y[i]*X[i] - (X[i]*np.exp(omiga.dot(X[i])))/(1 + np.exp(omiga.dot(X[i])))
        
    return re

#### 梯度下降法

In [9]:
#    要求对数似然函数的极大值，因此，按照梯度方向更新即可
#    因为梯度方向时函数上升最快的方向
#    梯度下降法
def gradient_descent(omiga, X, y, kexin, a):
    '''
    梯度下降法
    omiga: 扩充的参数向量
    X: 扩充的特征矩阵吧
    y: label
    kexin: 计算精度
    a: 步长
    return : 另目标函数最大的参数向量omiga
    '''
    while True:
        
        fk0 = L_omiga(omiga, X, y)
        #    求梯度
        gk = gradient_L_omiga(omiga, X, y)

        if np.linalg.norm(gk) < kexin :
            return omiga
        #    更新搜索方向，因为为梯度上升 所以同梯度的方向
        pk = gk
        #    更新omiga
        omiga += a*pk
        fk1 = L_omiga(omiga, X, y)
        if np.linalg.norm(fk1-fk0) < kexin or np.linalg.norm(omiga-gk) < kexin:
            return omiga   

In [10]:
omiga = np.zeros((1, X_train.shape[1]))        
omiga = gradient_descent(omiga, X_train, y_train, 0.1, 0.001)
omiga

array([[-0.18169974, -0.72017944,  1.1221241 ,  0.46584384, -0.12220339]])

#### 交叉验证 

In [11]:
predict = []
for i in range(len(X_test)):
    
    predict.append(1 if y_is_1(omiga, X_test[i]) > y_is_0(omiga, X_test[i]) else 0)
correct = [1 if i==j else 0 for (i,j) in zip(predict, y_test)]
print('交叉验证正确率: ', correct.count(1)/len(correct))

交叉验证正确率:  1.0
